In [18]:
import os
import json
import cv2
import random
import time

In [19]:


from PIL import Image


In [20]:
import shutil

directory_labels = os.fsencode("C:\\Users\\ganes\\Dataset_main\\val\\labels") #absolute path to the directory where all labels are stored
directory_images = os.fsencode("C:\\Users\\ganes\\Dataset_main\\val\\images") #absolute path to the directory where all images are stored
directory_unlabelled_images = os.fsencode("C:\\Users\\ganes\\Dataset_main\\unlabeled")#absolute path to the directory where unlabelled images are to be stored

not_labelled_files = 0
for file in os.listdir(directory_labels): #Read from the directory where the labels are stored
     filename = os.fsdecode(file)
     if filename.endswith(".txt"):
        yolo_annotation_path = (os.path.join(directory_labels.decode("utf-8"), filename))
        base=os.path.basename(yolo_annotation_path)
        file_name_without_ext = os.path.splitext(base)[0] # name of the file without the extension
        img_path = os.path.join(directory_images.decode("utf-8"), file_name_without_ext+ "." + 'jpg')
        
        filesize = os.path.getsize(yolo_annotation_path)
        if filesize == 0: #chek if the label file (.txt) is empty - that is it has no labels
            UnlabeledImg_path = os.path.join(directory_unlabelled_images.decode("utf-8"), file_name_without_ext+ "." + 'jpg')
            not_labelled_files = not_labelled_files +1
            
            shutil.move(img_path, UnlabeledImg_path) #remove the corresponding image to another directory
print("The number of unlabelled files is: ", not_labelled_files)

The number of unlabelled files is:  0


In [21]:
#### This is where the conversion process starts from YOLO to COCO format

# Category file, one category per line
yolo_format_classes_path = 'C:\\Users\\ganes\\Dataset_main\obj.names'
# Write the category according to your own data set. 

#Read the categories file and extract all categories
with open(yolo_format_classes_path,'r') as f1:
    lines1 = f1.readlines()
categories = []
for j,label in enumerate(lines1):
    label = label.strip()
    categories.append({'id':j+1,'name':label,'supercategory': label})
    
write_json_context = dict()
write_json_context['info'] = {'description': '', 'url': '', 'version': '', 'year': 2021, 'contributor': '', 'date_created': '2021-02-12 11:00:08.5'}
write_json_context['licenses'] = [{'id': 1, 'name': None, 'url': None}]
write_json_context['categories'] = categories
write_json_context['images'] = []
write_json_context['annotations'] = []

In [22]:
# Read the YOLO formatted label files (.txt) to extarct bounding box information and store in COCO format

#Read the label files (.txt) to extarct bounding box information and store in COCO format
directory_labels = os.fsencode("C:\\Users\\ganes\\Dataset_main\\val\\labels")
#directory_images = os.fsencode("/home/jyoti/Desktop/csc8800/datasets/DoorDetectDataset/test")
directory_images = os.fsencode("C:\\Users\\ganes\\Dataset_main\\val\\images")

file_number = 1
num_bboxes = 1
count=1
for file in os.listdir(directory_images):
    filename = os.fsdecode(file)
    if filename.endswith(".jpg"):
        img_path = (os.path.join(directory_images.decode("utf-8"), filename))
        base=os.path.basename(img_path)
        file_name_without_ext = os.path.splitext(base)[0] # name of the file without the extension
        yolo_annotation_path  = os.path.join(directory_labels.decode("utf-8"), file_name_without_ext+ "." + 'txt')
        img_name = os.path.basename(img_path) # name of the file without the extension
        img_context = {}
        height,width = cv2.imread(img_path).shape[:2]
        img_context['file_name'] = img_name
        img_context['height'] = height
        img_context['width'] = width
        img_context['date_captured'] = '2021-02-12 11:00:08.5'
        img_context['id'] = file_number # image id
        img_context['license'] = 1
        img_context['coco_url'] =''
        img_context['flickr_url'] = ''
        write_json_context['images'].append(img_context)
        
        with open(yolo_annotation_path,'r') as f2:
            lines2 = f2.readlines() 

        for i,line in enumerate(lines2): # for loop runs for number of annotations labelled in an image
            line = line.split(' ')
            bbox_dict = {}
            class_id, x_yolo,y_yolo,width_yolo,height_yolo= line[0:]
            x_yolo,y_yolo,width_yolo,height_yolo,class_id= float(x_yolo),float(y_yolo),float(width_yolo),float(height_yolo),int(class_id)
            bbox_dict['id'] = num_bboxes
            bbox_dict['image_id'] = file_number
            bbox_dict['category_id'] = class_id+1
            bbox_dict['iscrowd'] = 0 # There is an explanation before
            h,w = abs(height_yolo*height),abs(width_yolo*width)
            bbox_dict['area']  = h * w
            x_coco = round(x_yolo*width -(w/2))
            y_coco = round(y_yolo*height -(h/2))
            if x_coco <0: #check if x_coco extends out of the image boundaries
                x_coco = 1
            if y_coco <0: #check if y_coco extends out of the image boundaries
                y_coco = 1
            bbox_dict['bbox'] = [x_coco,y_coco,w,h]
            count=count+1
            bbox_dict['segmentation'] = [[x_coco,y_coco,x_coco+w,y_coco, x_coco+w, y_coco+h, x_coco, y_coco+h]]
            write_json_context['annotations'].append(bbox_dict)
            num_bboxes+=1
            #print(img_name)
            #print(x_coco,y_coco,w,h)
            count=count+1
            print(count)
            with open(r'C:\\Users\\ganes\\Dataset_main\\val\\labels_coco\\'+file_name_without_ext+'.txt', 'w') as fp:
                fp.write(str(x_coco)+' '+str(y_coco)+' '+str(w)+' '+str(h))
                pass
            
        file_number = file_number+1
        continue
    else:
        continue



        
 # Finally done, save!
#coco_format_save_path = '/home/jyoti/Desktop/csc8800/datasets/DoorDetectDataset/test.json'
#coco_format_save_path = 'C:\\Users\\ganes\\Dataset_main\\train\\train.json'
#with open(coco_format_save_path,'w') as fw:
 #   json.dump(write_json_context,fw) 
    

3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
51
53
55
57
59
61
63
65
67
69
71
73
75
77
79
81
83
85
87
89
91
93
95
97
99
101
103
105
107
109
111
113
115
117
119
121
123
125
127
129
131
133
135
137
139
141
143
145
147
149
151
153
155
157
159
161
163
165
167
169
171
173
175
177
179
181
183
185
187
189
191
193
195
197
199
201
203
205
207
209
211
213
215
217
219
221
223
225
227
229
231
233
235
237
239
241
243
245
247
249
251
253
255
257
259
261
263
265
267
269
271
273
275
277
279
281
283
285
287
289
291
293
295
297
299
301
303
305
307
309
311
313
315
317
319
321
323
325
327
329
331
333
335
337
339
341
343
345
347
349
351
353
355
357
359
361
363
365
367
369
371
373
375
377
379
381
383
385
387
389
391
393
395
397
399
401
403
405
407
409
411
413
415
417
419
421
423
425
427
429
431
433
435
437
439
441
443
445
447
449
451
453
455
457
459
461
463
465
467
469
471
473
475
477
479
481
483
485
487
489
491
493
495
497
499
501
503
505
507
509
511
513
515
517
519
521
523
525
527
5

3503
3505
3507
3509
3511
3513
3515
3517
3519
3521
3523
3525
3527
3529
3531
3533
3535
3537
3539
3541
3543
3545
3547
3549
3551
3553
3555
3557
3559
3561
3563
3565
3567
3569
3571
3573
3575
3577
3579
3581
3583
3585
3587
3589
3591
3593
3595
3597
3599
3601
3603
3605
3607
3609
3611
3613
3615
3617
3619
3621
3623
3625
3627
3629
3631
3633
3635
3637
3639
3641
3643
3645
3647
3649
3651
3653
3655
3657
3659
3661
3663
3665
3667
3669
3671
3673
3675
3677
3679
3681
3683
3685
3687
3689
3691
3693
3695
3697
3699
3701
3703
3705
3707
3709
3711
3713
3715
3717
3719
3721
3723
3725
3727
3729
3731
3733
3735
3737
3739
3741
3743
3745
3747
3749
3751
3753
3755
3757
3759
3761
3763
3765
3767
3769
3771
3773
3775
3777
3779
3781
3783
3785
3787
3789
3791
3793
3795
3797
3799
3801
3803
3805
3807
3809
3811
3813
3815
3817
3819
3821
3823
3825
3827
3829
3831
3833
3835
3837
3839
3841
3843
3845
3847
3849
3851
3853
3855
3857
3859
3861
3863
3865
3867
3869
3871
3873
3875
3877
3879
3881
3883
3885
3887
3889
3891
3893
3895
3897
3899
3901
